# Interactive basis plaything

Allows you to interactively move the basis points (black dots) like in the animation below.

![morphing animation](morphing.gif)

In [2]:
import bqplot
import numpy as np
from IPython.display import display
from ipywidgets import Layout

%matplotlib notebook

from madminer.utils.morphing import PhysicsMorpher as Morpher

## Settings

In [3]:
n_bases = 1
n_resolution = 50

## Preparation

In [4]:
morpher = Morpher(
    parameter_max_power=[4, 4],
    parameter_range=[(-1.0, 1.0), (-1.0, 1.0)],
)
morpher.find_components(max_overall_power=4)

In [5]:
xi = np.linspace(-1.0, 1.0, n_resolution)
yi = np.linspace(-1.0, 1.0, n_resolution)
xx, yy = np.meshgrid(xi, yi)
xx = xx.reshape((-1, 1))
yy = yy.reshape((-1, 1))
theta_evaluation = np.hstack([xx, yy])

x_updated = False
y_updated = False

## Basis evaluation

In [6]:
def evaluate_basis(basis=None):
    # Optimization
    if basis is None:
        basis = morpher.optimize_basis()

    # Evaluate basis
    squared_weights = []

    for theta in theta_evaluation:
        wi = morpher.calculate_morphing_weights(theta, basis)
        squared_weights.append(np.sum(wi * wi) ** 0.5)

    squared_weights = np.array(squared_weights).reshape((n_resolution, n_resolution))

    return basis, squared_weights

## Initial set up

In [7]:
basis, squared_weights = evaluate_basis(None)

## Interactive tool

In [8]:
def update(change):
    global basis, squared_weights, x_updated, y_updated

    variable = change["name"]
    values = change["new"]

    if variable == "x":
        basis[:, 0] = values
        x_updated = True
    elif variable == "y":
        basis[:, 1] = values
        y_updated = True

    if x_updated and y_updated:
        basis, squared_weights = evaluate_basis(basis)
        heat.color = np.log(squared_weights) / np.log(10)

        x_updated = False
        y_updated = False

In [9]:
x_sc = bqplot.scales.LinearScale(min=-1.0, max=1.0)
y_sc = bqplot.scales.LinearScale(min=-1.0, max=1.0)
c_sc = bqplot.scales.ColorScale(min=0, max=1.0, scheme="YlOrRd")

heat = bqplot.GridHeatMap(
    color=np.log(squared_weights) / np.log(10),
    scales={"row": x_sc, "column": y_sc, "color": c_sc},
    row=yi,
    column=xi,
    stroke=None,
)

scatter = bqplot.Scatter(
    colors=["black"] * len(basis[:, 0]),
    x=basis[:, 0],
    y=basis[:, 1],
    scales={"x": x_sc, "y": y_sc},
)

ax_x = bqplot.Axis(scale=x_sc, label="fW")
ax_y = bqplot.Axis(scale=y_sc, orientation="vertical", label="fWW")
ax_c = bqplot.ColorAxis(scale=c_sc, orientation="vertical", side="right")

fig = bqplot.Figure(
    marks=[heat, scatter],
    axes=[ax_x, ax_y, ax_c],
    layout=Layout(width="600px", height="600px"),
)

display(fig)

scatter.observe(update, ["y", "x"])
scatter.enable_move = True

RmlndXJlKGF4ZXM9W0F4aXMobGFiZWw9dSdmVycsIHNjYWxlPUxpbmVhclNjYWxlKG1heD0xLjAsIG1pbj0tMS4wKSksIEF4aXMobGFiZWw9dSdmV1cnLCBvcmllbnRhdGlvbj0ndmVydGljYWzigKY=
